# M2177.004300 Deep Learning Assignment #1<br> Part 1-3. Training Multi-Layer RNN (NumPy)

Copyright (C) Data Science & AI Laboratory, Seoul National University. This material is for educational uses only. Some contents are based on the material provided by other paper/book authors and may be copyrighted by them. Written by Jieun Byeon, September 2025

**For understanding of this work, please carefully look at given PDF file.**

Now, you're going to learn how to implement vanilla RNN using NumPy. <br>


![RNN](imgs/rnn.PNG)
![the unfolding in time of the computation involved in its forward computation](imgs/unfold.jpg) <br>
$x_t$: input at time step $t$<br>
$s_t$: hidden state at time step $t$<br>
$o_t$: output at time step $t$<br>
$W, U, V$: weight for input, hidden state, and output<br>
$s_t = tanh(Ux_t + Ws_{t-1})$<br>
$o_t = softmax(Vs_t)$<br>

In [3]:
import numpy as np

seed = 2177
np.random.seed(seed)

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
%pwd

'/content'

In [6]:
# Assighment1 경로로 이동
%cd /content/drive/MyDrive/Assignment1

/content/drive/MyDrive/Assignment1


##  Vanilla RNN: step forward and backward

In this section, you will implement step forward and backward passes for **a single timestep** of a vanilla recurrent neural network. Using the code provided as guidance, complete the functions `rnn_step_forward` and `rnn_step_backward`. just write the code in whatever way you find most clear.

When you are done, run the following to check your implementations.

### <a name="1"></a> 1. single timestep forward

In [7]:
def getdata_rnn_step_forward():
	np.random.seed(seed)

	N, D, H = 3, 5, 4
	x = np.random.randn(N, D) # current input
	prev_h = np.random.randn(N, H) # previous hidden state
	Wx = np.random.randn(D, H) # input -> hidden weights
	Wh = np.random.randn(H, H) # hidden -> hidden weights
	b = np.random.randn(H) # biases

	expt_next_h = np.asarray([ # expected next hidden state
		[-0.99921173, -0.99967951,  0.39127099, -0.93436299],
	 	[ 0.84348286,  0.99996526, -0.9978802,   0.99996645],
	  [-0.94481752, -0.71940178,  0.99994009, -0.64806562]])

	return x, prev_h, Wx, Wh, b, expt_next_h


In [8]:
#------------------------------------------------------
# vanilla rnn step forward
#------------------------------------------------------
def rnn_step_forward(x, prev_h, Wx, Wh, b):
    """
    Run the forward pass for a single timestep of a vanilla RNN that uses a tanh
    activation function.

    The input data has dimension D, the hidden state has dimension H, and we use
    a minibatch size of N.

    Inputs:
    - x: Input data for this timestep, of shape (N, D).
    - prev_h: Hidden state from previous timestep, of shape (N, H)
    - Wx: Weight matrix for input-to-hidden connections, of shape (D, H)
    - Wh: Weight matrix for hidden-to-hidden connections, of shape (H, H)
    - b: Biases of shape (H,)

    Returns a tuple of:
    - next_h: Next hidden state, of shape (N, H)
    - cache: Tuple of values needed for the backward pass.
    """
    next_h, cache = None, None
    ##########################################################################
    # TODO: Implement a single forward step for the vanilla RNN. Store the next  #
    # hidden state and any values you need for the backward pass in the next_h   #
    # and cache variables respectively.                                          #
    ##########################################################################

    a = x.dot(Wx) + prev_h.dot(Wh) + b
    next_h = np.tanh(a)
    cache = (x, prev_h, Wx, Wh, a)
    return next_h, cache

    ##########################################################################
    #                               END OF YOUR CODE                             #
    ##########################################################################
    return next_h, cache

In [9]:
def rel_error(x, y):
    """ returns relative error """
    return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

In [10]:
# implement rnn_step_forward
# errors should be less than 1e-8
x, prev_h, Wx, Wh, b, expected_next_h = getdata_rnn_step_forward()

next_h, _ = rnn_step_forward(x, prev_h, Wx, Wh, b)

print('next_h error: ', rel_error(expected_next_h, next_h))

next_h error:  2.5562554071588464e-09


### <a name="1"></a> 2. single timestep backward

In [11]:
# function for numerical gradient check
def eval_numerical_gradient_array(f, x, df, h=1e-5):
    """
    Evaluate a numeric gradient for a function that accepts a numpy
    array and returns a numpy array.
    """
    grad = np.zeros_like(x)
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    while not it.finished:
        ix = it.multi_index

        oldval = x[ix]
        x[ix] = oldval + h
        pos = f(x).copy()
        x[ix] = oldval - h
        neg = f(x).copy()
        x[ix] = oldval

        grad[ix] = np.sum((pos - neg) * df) / (2 * h)
        it.iternext()
    return grad

In [12]:
# get numerical gradient for rnn_step_backward
# ground truth for rnn_step_backward
def getdata_rnn_step_backward(x,h,Wx,Wh,b,dnext_h):

	fx  = lambda x: rnn_step_forward(x, h, Wx, Wh, b)[0]
	fh  = lambda prev_h: rnn_step_forward(x, h, Wx, Wh, b)[0]
	fWx = lambda Wx: rnn_step_forward(x, h, Wx, Wh, b)[0]
	fWh = lambda Wh: rnn_step_forward(x, h, Wx, Wh, b)[0]
	fb  = lambda b: rnn_step_forward(x, h, Wx, Wh, b)[0]

	dx_num = eval_numerical_gradient_array(fx, x, dnext_h)
	dprev_h_num = eval_numerical_gradient_array(fh, h, dnext_h)
	dWx_num = eval_numerical_gradient_array(fWx, Wx, dnext_h)
	dWh_num = eval_numerical_gradient_array(fWh, Wh, dnext_h)
	db_num = eval_numerical_gradient_array(fb, b, dnext_h)

	return dx_num, dprev_h_num, dWx_num, dWh_num, db_num

In [22]:
#------------------------------------------------------
# vanilla rnn step backward
#------------------------------------------------------
def rnn_step_backward(dnext_h, cache):
    """
    Backward pass for a single timestep of a vanilla RNN.

    Inputs:
    - dnext_h: Gradient of loss with respect to next hidden state
    - cache: Cache object from the forward pass

    Returns a tuple of:
    - dx: Gradients of input data, of shape (N, D)
    - dprev_h: Gradients of previous hidden state, of shape (N, H)
    - dWx: Gradients of input-to-hidden weights, of shape (D, H)
    - dWh: Gradients of hidden-to-hidden weights, of shape (H, H)
    - db: Gradients of bias vector, of shape (H,)
    """
    dx, dprev_h, dWx, dWh, db = None, None, None, None, None
    ##########################################################################
    # TODO: Implement the backward pass for a single step of a vanilla RNN.      #
    #                                                                            #
    # HINT: For the tanh function, you can compute the local derivative in terms #
    # of the output value from tanh.                                             #
    ##########################################################################

    x, prev_h, Wx, Wh, a = cache
    dtanh = 1-np.tanh(a)**2
    da = dnext_h * dtanh

    dx = da.dot(Wx.T)
    dprev_h = da.dot(Wh.T)
    dWx = x.T.dot(da)
    dWh = prev_h.T.dot(da)
    db = np.sum(da, axis=0)
    ##########################################################################
    #                               END OF YOUR CODE                             #
    ##########################################################################
    return dx, dprev_h, dWx, dWh, db

In [14]:
x, h, Wx, Wh, b, expected_next_h = getdata_rnn_step_forward()
out, cache = rnn_step_forward(x, h, Wx, Wh, b)
dnext_h = np.random.randn(*out.shape)
dx_num, dprev_h_num, dWx_num, dWh_num, db_num = getdata_rnn_step_backward(x,h,Wx,Wh,b,dnext_h)

dx, dprev_h, dWx, dWh, db = rnn_step_backward(dnext_h, cache)

print('dx error     : ', rel_error(dx_num, dx))
print('dprev_h error: ', rel_error(dprev_h_num, dprev_h))
print('dWx error    : ', rel_error(dWx_num, dWx))
print('dWh error    : ', rel_error(dWh_num, dWh))
print('db error     : ', rel_error(db_num, db))

dx error     :  6.111935337890625e-10
dprev_h error:  7.699207487627533e-11
dWx error    :  4.917216370202925e-10
dWh error    :  9.995079967788294e-11
db error     :  3.033863959395676e-11


## Vanilla RNN: forward and backward

Combining the single timestep forward and backward passes for vanilla RNN, you will implement a vanilla RNN that process **an entire sequence**. Using the code provided as guidance, complete the functions `rnn_forward` and `rnn_backward`.

When you are done, run the following to check your implementations.

### <a name="1"></a> 3. forward pass for an entire sequence

In [17]:
def getdata_rnn_forward():
	np.random.seed(seed)

	N, D, T, H = 2,3,4,5
	x = np.random.randn(N, T, D)
	h0 = np.random.randn(N, H)
	Wx = np.random.randn(D, H)
	Wh = np.random.randn(H, H)
	b = np.random.randn(H)

	expt_next_h = np.asarray([
	[[ 0.79899136, -0.90076473, -0.69325878, -0.99991011,  0.92991908],
   [-0.04474799, -0.99999994, -0.72167573, -0.99942462, -0.98397185],
   [ 0.98674954, -0.74668554, -0.30836793, -0.87580427, -0.25076433],
   [ 0.99999994,  0.46495278, -0.6291276 ,  0.44811995, -0.91013617]],

 	[[-0.57789921, -0.10875688, -0.99049558, -0.58448393,  0.76942269],
   [-0.05646372, -0.99855467, -0.827688  , -0.65262183, -0.98211725],
   [ 0.89687939,  0.99998112, -0.99999517,  0.66932722,  0.99952606],
   [-0.97608409, -0.64972242, -0.99987169, -0.99747724,  0.99962792]]])

	return x, h0, Wx, Wh, b, expt_next_h

In [23]:
#------------------------------------------------------
# vanilla rnn forward
#------------------------------------------------------
def rnn_forward(x, h0, Wx, Wh, b):
    """
    Run a vanilla RNN forward on an entire sequence of data. We assume an input
    sequence composed of T vectors, each of dimension D. The RNN uses a hidden
    size of H, and we work over a minibatch containing N sequences. After running
    the RNN forward, we return the hidden states for all timesteps.

    Inputs:
    - x: Input data for the entire timeseries, of shape (N, T, D).
    - h0: Initial hidden state, of shape (N, H)
    - Wx: Weight matrix for input-to-hidden connections, of shape (D, H)
    - Wh: Weight matrix for hidden-to-hidden connections, of shape (H, H)
    - b: Biases of shape (H,)

    Returns a tuple of:
    - h: Hidden states for the entire timeseries, of shape (N, T, H).
    - cache: Values needed in the backward pass
    """
    h, cache = None, None
    ##########################################################################
    # TODO: Implement forward pass for a vanilla RNN running on a sequence of    #
    # input data. You should use the rnn_step_forward function that you defined  #
    # above.                                                                     #
    ##########################################################################
    N, T, D = x.shape
    H = h0.shape[1]

    h = np.zeros((N, T, H), dtype=x.dtype)
    cache = []
    prev_h = h0

    for t in range(T):
        xt = x[:, t, :]
        next_h, step_cache = rnn_step_forward(xt, prev_h, Wx, Wh, b)
        h[:, t, :] = next_h
        cache.append(step_cache)
        prev_h = next_h

    return h, cache

    ##########################################################################
    #                               END OF YOUR CODE                             #
    ##########################################################################
    return h, cache

In [24]:
# implement rnn_forward
# errors should be less than 1e-7
x,h0,Wx,Wh,b, expected_h = getdata_rnn_forward()

h, _ = rnn_forward(x, h0, Wx, Wh, b)

print('h error: ', rel_error(expected_h, h))

h error:  2.1007953765777348e-08


### <a name="1"></a> 4. backward pass for an entire sequence

In [25]:
def getdata_rnn_backward(x,h0,Wx,Wh,b,dout):
	fx = lambda x: rnn_forward(x, h0, Wx, Wh, b)[0]
	fh0 = lambda h0: rnn_forward(x, h0, Wx, Wh, b)[0]
	fWx = lambda Wx: rnn_forward(x, h0, Wx, Wh, b)[0]
	fWh = lambda Wh: rnn_forward(x, h0, Wx, Wh, b)[0]
	fb = lambda b: rnn_forward(x, h0, Wx, Wh, b)[0]

	dx_num = eval_numerical_gradient_array(fx, x, dout)
	dh0_num = eval_numerical_gradient_array(fh0, h0, dout)
	dWx_num = eval_numerical_gradient_array(fWx, Wx, dout)
	dWh_num = eval_numerical_gradient_array(fWh, Wh, dout)
	db_num = eval_numerical_gradient_array(fb, b, dout)

	return dx_num, dh0_num, dWx_num, dWh_num, db_num

In [27]:
#------------------------------------------------------
# vanilla rnn backward
#------------------------------------------------------
def rnn_backward(dh, cache):
    """
    Compute the backward pass for a vanilla RNN over an entire sequence of data.

    Inputs:
    - dh: Upstream gradients of all hidden states, of shape (N, T, H)

    Returns a tuple of:
    - dx: Gradient of inputs, of shape (N, T, D)
    - dh0: Gradient of initial hidden state, of shape (N, H)
    - dWx: Gradient of input-to-hidden weights, of shape (D, H)
    - dWh: Gradient of hidden-to-hidden weights, of shape (H, H)
    - db: Gradient of biases, of shape (H,)
    """
    dx, dh0, dWx, dWh, db = None, None, None, None, None

    ##########################################################################
    # TODO: Implement the backward pass for a vanilla RNN running an entire      #
    # sequence of data. You should use the rnn_step_backward function that you   #
    # defined above.                                                             #
    ##########################################################################
    T = len(cache)
    x0, prev_h0, Wx0, Wh0, a0 = cache[0]
    N, D = x0.shape
    H = prev_h0.shape[1]

    dx  = np.zeros((N, T, D), dtype=x0.dtype)
    dWx = np.zeros_like(Wx0)
    dWh = np.zeros_like(Wh0)
    db  = np.zeros((H,), dtype=x0.dtype)

    dprev_h = np.zeros((N, H), dtype=x0.dtype)

    for t in reversed(range(T)):
        dnext_h = dh[:, t, :] + dprev_h
        dx_t, dprev_h, dWx_t, dWh_t, db_t = rnn_step_backward(dnext_h, cache[t])

        dx[:, t, :] += dx_t
        dWx += dWx_t
        dWh += dWh_t
        db  += db_t

    dh0 = dprev_h
    return dx, dh0, dWx, dWh, db

    ##########################################################################
    #                               END OF YOUR CODE                             #
    ##########################################################################
    return dx, dh0, dWx, dWh, db

In [28]:
x,h0,Wx,Wh,b, expected_h = getdata_rnn_forward()
out, cache = rnn_forward(x, h0, Wx, Wh, b)
dout = np.random.randn(*out.shape)
dx_num, dh0_num, dWx_num, dWh_num, db_num = getdata_rnn_backward(x,h0,Wx,Wh,b,dout)

dx, dh0, dWx, dWh, db = rnn_backward(dout, cache)

print('dx error: ', rel_error(dx_num, dx))
print('dh0 error: ', rel_error(dh0_num, dh0))
print('dWx error: ', rel_error(dWx_num, dWx))
print('dWh error: ', rel_error(dWh_num, dWh))
print('db error: ', rel_error(db_num, db))

dx error:  1.9080063186473363e-08
dh0 error:  7.196103315383523e-10
dWx error:  1.3418004368964638e-10
dWh error:  5.665377918363601e-09
db error:  8.441613976110239e-10
